## 设置学习率

- 在神经网络模型中，通常使用标准的随机梯度下降算法更新参数，学习率代表参数更新幅度的大小，即步长。
- 当学习率最优时，模型的有效容量最大，最终能达到的效果最好。
- 设置合适的学习率需要大量的实验和调参经验：
    - 学习率不是越小越好。学习率越小，损失函数的变化速度越慢，意味着需要花费更长的时间进行收敛。
    - 学习率不是越大越好。只根据总样本集中的一个批次计算梯度，抽样误差会导致计算出的梯度不是全局最优的方向，且存在波动。在接近最优解时，过大的学习率会导致参数在最优解附近震荡，损失难以收敛。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import paddle
from paddle.nn import Conv2D, MaxPool2D, Linear
import paddle.nn.functional as F

from PIL import Image
import os
import random
import gzip
import json

In [2]:
# 构建一个加载数据的迭代器
class MnistDataset(paddle.io.Dataset):
    def __init__(self, mode):
        # 读取原始数据
        datafile = '../datasets/mnist.json.gz'
        data = json.load(gzip.open(datafile))
        
        # 划分训练集、验证集和测试集
        train_set, valid_set, test_set = data
        
        # 数据集相关参数：图片宽、高度
        self.IMG_ROWS = 28
        self.IMG_COLS = 28
        
        # 拆分数据和标签
        if mode=='train':
            imgs, labels = train_set[0], train_set[1]
        elif mode=='valid':
            imgs, labels = valid_set[0], valid_set[1]
        elif mode=='test':
            imgs, labels = test_set[0], test_set[1]
        else:
            raise Exception("Mode can only be one of ['train', 'valid', 'test']")
                            
        # 校验数据
        imgs_length = len(imgs)
        assert len(imgs) == len(labels), \
            'Length of imgs({}) should be the same as labels({})'.format(len(imgs), len(labels))
                            
        self.imgs = imgs
        self.labels = labels
    
    def __getitem__(self, idx):
        img = np.reshape(self.imgs[idx], [1, self.IMG_ROWS, self.IMG_COLS]).astype('float32')
        label = np.reshape(self.labels[idx], [1]).astype('int64')
        
        return img, label
    
    def __len__(self):
        return len(self.imgs)

In [3]:
# 使用训练模式，迭代器每次迭代只返回batch=1的数据
train_dataset = MnistDataset(mode='train')

# 使用paddle.io.DataLoader，返回的是一个批次数据迭代器，并且是异步的
train_loader = paddle.io.DataLoader(train_dataset, batch_size=100, shuffle=True, drop_last=True)

# 加载验证数据集
valid_dataset = MnistDataset(mode='valid')
valid_loader = paddle.io.DataLoader(valid_dataset, batch_size=128, drop_last=True)

In [4]:
# 定义多层卷积神经网络
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()

        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义一层全连接层，输出维度是10
        self.fc = Linear(in_features=980, out_features=10)
        
    # 定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连接层计算最终输出
    # 卷积层激活函数使用Relu，全连接层激活函数使用softmax
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], 980])
        x = self.fc(x)
        return x

In [5]:
# 修改计算损失函数
def evaluation(model, datasets):
    model.eval()
    
    acc_set = list()
    for batch_id, data in enumerate(datasets()):
        images, labels = data
        images = paddle.to_tensor(images)
        labels = paddle.to_tensor(labels)
        pred = model(images)
        acc = paddle.metric.accuracy(input=pred, label=labels)
        acc_set.extend(acc.numpy())
    
    # 计算多个批次的准确率
    acc_val_mean = np.array(acc_set).mean()
    return acc_val_mean

In [16]:
""" 学习率的主流优化算法

- SGD: 随机梯度下降算法，每次训练少量数据，抽样偏差导致的参数收敛过程中震荡。
- Momentum: 每个批次的数据含有抽样误差，导致梯度更新的方向波动较大。引入物理“动量”的概念，累积速度，减少震荡，使参数更新的方向更稳定。
- AdaGrad: 根据不同参数距离最优解的远近，动态调整学习率。学习率逐渐下降，依据各参数变化大小调整学习率。
- Adam: 由于动量和自适应学习率两个优化思路是正交的，因此可以将这两两个思路结合起来，这就是当前广泛应用的算法。

每种优化算法都有特定的参数设置，理论最合理未必在具体案例中最有效，所以模型调参是非常必要的。
"""


def train(model):
    model.train()
    
    # 四种不同的优化算法
    # opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())
    # opt = paddle.optimizer.Momentum(learning_rate=0.01, parameters=model.parameters())
    # opt = paddle.optimizer.Adagrad(learning_rate=0.01, parameters=model.parameters())
    opt = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
    
    EPOCH_NUM = 3
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            # 准备数据
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)
            # 前向计算的过程
            predicts = model(images)
            
            # 计算损失，使用交叉熵损失函数，取一个批次样本损失的平均值
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            
            # 每训练了200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))
            
            # 后向传播，更新参数的过程
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.step()
            # 清除梯度
            opt.clear_grad()
            
        acc_train_mean = evaluation(model, train_loader)
        acc_valid_mean = evaluation(model, valid_loader)
        print('train_acc: {}, valid acc: {}'.format(acc_train_mean, acc_valid_mean))
        
    #保存模型参数
    paddle.save(model.state_dict(), '../models/mnist.pdparams')

In [17]:
# Init Model
model = MNIST()

# Start training
train(model)

epoch: 0, batch: 0, loss is: [4.231844]
epoch: 0, batch: 200, loss is: [0.20475605]
epoch: 0, batch: 400, loss is: [0.24968132]
train_acc: 0.9730600714683533, valid acc: 0.9748597741127014
epoch: 1, batch: 0, loss is: [0.12568131]
epoch: 1, batch: 200, loss is: [0.10303674]
epoch: 1, batch: 400, loss is: [0.05452614]
train_acc: 0.9809000492095947, valid acc: 0.9805688858032227
epoch: 2, batch: 0, loss is: [0.01553402]
epoch: 2, batch: 200, loss is: [0.05573765]
epoch: 2, batch: 400, loss is: [0.11679833]
train_acc: 0.9834599494934082, valid acc: 0.9803686141967773


In [18]:
# Load a local image, and convert format to model
def load_image(img_path):
    img = Image.open(img_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).reshape(1, 1, 28, 28).astype(np.float32)
    # 图像归一化
    img = 1.0 - img / 255
    return img

In [19]:
model = MNIST()
model_params_path = '../models/mnist.pdparams'
img_path = '../datasets/mnist_example_0.jpg'

# Load model's params
param_dict = paddle.load(model_params_path)
model.load_dict(param_dict)

model.eval()
tensor_img = load_image(img_path)

# 模型返回10个分类标签的对应概率
results = model(paddle.to_tensor(tensor_img))

# 取出概率最大的标签作为预测输出
label = np.argsort(results.numpy())

print('识别结果：', label[0][-1])

识别结果： 0
